## Build Data Generators

In [44]:
import numpy as np
from keras import backend as K
from keras import callbacks
K.set_image_dim_ordering('th')
from time import time

from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
import PIL

from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications.imagenet_utils import decode_predictions
from keras.optimizers import SGD, Adadelta, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard

In [2]:
def build_flow(directory, target_size, batch_size,validation_split):
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split = validation_split)
    
    train_datagen.mean = np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1, 1, 3)

    train_generator = train_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True) # set as training data

    validation_generator = train_datagen.flow_from_directory(
        directory, # same directory as training data
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False) # set as validation data
    
    return train_datagen, train_generator, validation_generator

In [3]:
data_dir = '../data/raw/normal'
target_size = (200,200)
batch_size = 8
validation_split = 0.2

In [8]:
train_datagen, train_generator, validation_generator = build_flow(data_dir,target_size,batch_size,validation_split)

Found 1740 images belonging to 29 classes.
Found 435 images belonging to 29 classes.


## Build class label mapping

In [9]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [3]:
data_dir = '../data/raw/normal'
target_size = (200,200)
batch_size = 8
validation_split = 0.2

In [8]:
train_datagen, train_generator, validation_generator = build_flow(data_dir,target_size,batch_size,validation_split)

Found 1740 images belonging to 29 classes.
Found 435 images belonging to 29 classes.


In [11]:
import pickle
pickle.dump(labels,open("../label_map.pickle","wb"))

## Transfer Learning with VGG16

Top layer is custom with 2 Dense layers and two dropouts. Also, the next top block of the vgg16 is unfrozen to train and update its weights.

In [12]:
# note we exclude the final dense layers and add one back below, we would retrain it ourselves
base = VGG16(weights='imagenet', include_top=False, input_shape=(3,200,200)) 
 
# Freeze convolutional layers
for layer in base.layers[:-4]:
    layer.trainable = False    
    
x = base.output
x = Dense(512, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', name='fc2')(x)
x = Dropout(0.5)(x)
x = Flatten()(x) # flatten from convolution tensor output 
predictions = Dense(29, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base.input, outputs=predictions)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 200, 200)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 200, 200)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 200, 200)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 100, 100)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 100, 100)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 100, 100)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 50, 50)       0         
__________

Callbacks for early stopping, and tensorboard logging

In [15]:
myCallback=[
    EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=0, mode='auto', 
                  baseline=None, restore_best_weights=True),
    TensorBoard(log_dir="../logs/vgg16_final2", batch_size=batch_size, write_graph=True,update_freq='epoch')
]

In [16]:
vgg16_sgd=model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 40, callbacks=myCallback)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
217/217 [==============================] - 34s 154ms/step - loss: 3.3060 - acc: 0.0870 - val_loss: 3.0681 - val_acc: 0.1991
Epoch 2/40
217/217 [==============================] - 24s 112ms/step - loss: 2.4017 - acc: 0.3739 - val_loss: 1.7803 - val_acc: 0.5621
Epoch 3/40
217/217 [==============================] - 24s 111ms/step - loss: 1.0400 - acc: 0.7316 - val_loss: 0.8183 - val_acc: 0.7588
Epoch 4/40
217/217 [==============================] - 24s 111ms/step - loss: 0.4603 - acc: 0.8819 - val_loss: 0.5067 - val_acc: 0.8337
Epoch 5/40
217/217 [==============================] - 24s 110ms/step - loss: 0.2675 - acc: 0.9332 - val_loss: 0.3667 - val_acc: 0.8829
Epoch 6/40
217/217 [==============================] - 24s 109ms/step - loss: 0.1824 - acc: 0.9516 - val_loss: 0.3281 - val_acc: 0.9016
Epoch 7/40
217/217 [==============================] - 23s 107ms/step - loss: 0.1367 - acc: 0.9672 - val_loss: 0.2560 - val_acc: 0.9274
Epoch 8

In [17]:
model.save("../vgg16_final2.h5")

## Transfer Learning with MobileNet

In [99]:
# note we exclude the final dense layers and add one back below, we would retrain it ourselves
mn_base = MobileNet(input_shape=(224,224,3), weights='imagenet', include_top=False) 
 
# Freeze convolutional layers
for layer in mn_base.layers[:-19]:
    layer.trainable = False    
    
x = mn_base.output
x = Dense(512, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', name='fc2')(x)
x = Dropout(0.5)(x)
x = Flatten()(x) # flatten from convolution tensor output 
predictions = Dense(29, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
mn_model = Model(inputs=mn_base.input, outputs=predictions)

In [100]:
mn_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', metrics=['accuracy'])

In [101]:
mn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [102]:
myCallback2=[
    EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='auto', 
                  baseline=None, restore_best_weights=True),
    TensorBoard(log_dir="../logs/mobilenet_v1_open3blk_sgd", batch_size=batch_size, write_graph=True,update_freq='epoch')
]

In [103]:
mn_target_size = (224,224)

train_datagen2, train_generator2, validation_generator2 = build_flow(data_dir,mn_target_size,batch_size,validation_split)

Found 1740 images belonging to 29 classes.
Found 435 images belonging to 29 classes.


In [ ]:
mn_sgd=mn_model.fit_generator(
    train_generator2,
    steps_per_epoch = train_generator2.samples // batch_size,
    validation_data = validation_generator2, 
    validation_steps = validation_generator2.samples // batch_size,
    epochs = 50, callbacks=myCallback2)

Epoch 1/50
217/217 [==============================] - 31s 141ms/step - loss: 4.8127 - acc: 0.0876 - val_loss: 2.9783 - val_acc: 0.1528
Epoch 2/50
217/217 [==============================] - 28s 130ms/step - loss: 2.8707 - acc: 0.2684 - val_loss: 2.3358 - val_acc: 0.3724
Epoch 3/50
217/217 [==============================] - 28s 130ms/step - loss: 1.8904 - acc: 0.4620 - val_loss: 1.8128 - val_acc: 0.5246
Epoch 4/50
217/217 [==============================] - 28s 130ms/step - loss: 1.2111 - acc: 0.6406 - val_loss: 1.5125 - val_acc: 0.6042
Epoch 5/50
217/217 [==============================] - 29s 133ms/step - loss: 0.9316 - acc: 0.7108 - val_loss: 1.1910 - val_acc: 0.7143
Epoch 6/50
217/217 [==============================] - 28s 130ms/step - loss: 0.7284 - acc: 0.7736 - val_loss: 1.0389 - val_acc: 0.7190
Epoch 7/50
217/217 [==============================] - 29s 132ms/step - loss: 0.5726 - acc: 0.8237 - val_loss: 0.9280 - val_acc: 0.7658
Epoch 8/50
172/217 [======================>.......] - E

In [106]:
mn_model.save("../mobilenet_v1.h5")

## Transfer Learning with MobileNet V2

In [117]:
# note we exclude the final dense layers and add one back below, we would retrain it ourselves
mnv2_base = MobileNetV2(input_shape= (224,224,3), weights='imagenet', include_top=False) 
 
# Freeze convolutional layers
for layer in mn_base.layers[:-19]:
    layer.trainable = False    
    
x = mnv2_base.output
x = Dense(512, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', name='fc2')(x)
x = Dropout(0.5)(x)
x = Flatten()(x) # flatten from convolution tensor output 
predictions = Dense(29, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
mnv2_model = Model(inputs=mnv2_base.input, outputs=predictions)

In [118]:
mnv2_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', metrics=['accuracy'])

In [119]:
mnv2_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_21[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [120]:
myCallback3=[
    EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='auto', 
                  baseline=None, restore_best_weights=True),
    TensorBoard(log_dir="../logs/mobilenet_v2_open3blk_sgd", batch_size=batch_size, write_graph=True,update_freq='epoch')
]

In [121]:
mnv2_target_size = (224,224)

train_datagen3, train_generator3, validation_generator3 = build_flow(data_dir,mnv2_target_size,batch_size,validation_split)

Found 1740 images belonging to 29 classes.
Found 435 images belonging to 29 classes.


In [124]:
mnv2_sgd=mnv2_model.fit_generator(
    train_generator3,
    steps_per_epoch = train_generator3.samples // batch_size,
    validation_data = validation_generator3, 
    validation_steps = validation_generator3.samples // batch_size,
    epochs = 50, callbacks=myCallback3)

Epoch 1/50
217/217 [==============================] - 40s 184ms/step - loss: 4.1628 - acc: 0.0737 - val_loss: 2.9133 - val_acc: 0.1850
Epoch 2/50
217/217 [==============================] - 30s 139ms/step - loss: 2.9169 - acc: 0.2085 - val_loss: 2.2786 - val_acc: 0.3841
Epoch 3/50
217/217 [==============================] - 30s 138ms/step - loss: 1.9819 - acc: 0.4176 - val_loss: 1.4512 - val_acc: 0.5738
Epoch 4/50
217/217 [==============================] - 30s 136ms/step - loss: 1.2890 - acc: 0.6014 - val_loss: 0.9545 - val_acc: 0.7307
Epoch 5/50
217/217 [==============================] - 30s 140ms/step - loss: 0.8860 - acc: 0.7177 - val_loss: 0.5786 - val_acc: 0.8454
Epoch 6/50
217/217 [==============================] - 30s 136ms/step - loss: 0.6252 - acc: 0.8036 - val_loss: 0.4463 - val_acc: 0.8665
Epoch 7/50
217/217 [==============================] - 30s 136ms/step - loss: 0.4689 - acc: 0.8502 - val_loss: 0.3197 - val_acc: 0.9087
Epoch 8/50
217/217 [==============================] - 3

In [126]:
mnv2_model.save("../mobilenet_v2.h5")

## Model Prediction testing with validation set

In [18]:
from keras.models import load_model

In [19]:
vgg = load_model("../vgg16_final2.h5")

In [42]:
import numpy as np

def predict_single(image):
    a=np.array(image,dtype='float32')
    a = np.flip(a,axis=2)
    a[:, :, 2] -= 123.68
    a[:, :, 1] -= 116.779
    a[:, :, 0] -= 103.939
    a /= 255
    a = a.transpose((2,0,1))
    a = np.expand_dims(a,axis=0)
    
    p=vgg.predict(a)
    select = np.argmax(p)
    
    return select

def validation_check(validation_generator):
    
    """
    Check Validation accuracy one more time
    """
    val_size = len(validation_generator.filenames)
    match = 0
    
    for i in range(val_size):
        y_true = validation_generator.classes[i]
        example = validation_generator.filenames[i]
        a=PIL.Image.open("../data/raw/normal/"+example).resize((200,200))
        y_pred = predict_single(a)
    
        if y_true==y_pred:
            match += 1
    
    return match/val_size

In [43]:
validation_check(validation_generator)

0.9701149425287356